Importing initial libraries nessescary for creating and running the ![model](path)

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import seaborn as sns

In [0]:
torch.__version__

In [0]:
import pandas as pd

file_path = "/dbfs/mnt/sid/d.csv"

df = pd.read_csv(file_path)

from ydata_profiling import ProfileReport
profile = ProfileReport(df, title="Thing", )
profile.to_notebook_iframe()

df.head(6)

In [0]:
df.isnull().sum()

In [0]:
import numpy as np
df["Outcome"]  = np.where(df["Outcome"] == 1, "Diabetic", "Non-diabetic")

df.head(5)

In [0]:
import seaborn as sns
sns.pairplot(df, hue = "Outcome")

In [0]:
df["Outcome"] = np.where(df["Outcome"] == "Diabetic", 1, 0)
df.head(5)

In [0]:
X = df.drop("Outcome", axis = 1).values
y = df["Outcome"].values

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [0]:
X_train = torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_test)  
y_train = torch.LongTensor(y_train)
y_test = torch.LongTensor(y_test) 

In [0]:
y_test

In [0]:
X_train

In [0]:
class ANN_Model(nn.Module):
    def __init__(self,input_features=8,hidden1=20,hidden2=20,out_features=2):
        super().__init__()
        self.f_connected1=nn.Linear(input_features,hidden1)
        self.f_connected2=nn.Linear(hidden1,hidden2)
        self.out=nn.Linear(hidden2,out_features)
        
    def forward(self,x):
        x=F.relu(self.f_connected1(x))
        x=F.relu(self.f_connected2(x))
        x=self.out(x)
        return x

In [0]:

torch.manual_seed(20)
model=ANN_Model()


In [0]:
model.parameters

In [0]:
loss_function=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model.parameters(),lr=0.01)

In [0]:
trainingcycles = 500
final_losses=[]
for trainingcycle in range(trainingcycles):
    trainingcycle = trainingcycle+1
    y_pred=model.forward(X_train)
    loss=loss_function(y_pred,y_train)
    final_losses.append(loss)
    print("Training Cycle number: {} and the loss : {}".format(trainingcycle,loss.item()))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

In [0]:
predictions=[]
with torch.no_grad():
    for i,data in enumerate(X_test):
        y_pred=model(data)
        predictions.append(y_pred.argmax().item())
        print(y_pred.argmax().item())

In [0]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,predictions)
print(cm)

In [0]:
import matplotlib.pyplot as plt
plt.figure(figsize=(10,6))
sns.heatmap(cm, annot=True)
plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')

In [0]:
from sklearn.metrics import accuracy_score
score=accuracy_score(y_test,predictions)
score

In [0]:
model.eval()